In [1]:
import pandas as pd
import numpy as np
import json
import ast

In [2]:
def clean_yelp(df1,df2):
    if type(df2) != None:
        final = pd.concat([df1,df2])
    else:
        final = df1
    final = final.drop_duplicates(subset = "id",keep = "first").reset_index(drop = True)
    first_cat = []
    second_cat = []
    third_cat = []
    for entry in range(len(final['categories'])):
        x = ast.literal_eval(final['categories'][entry])
        if len(x)>0:
            first_cat.append(x[0]['alias'])
        else:
            first_cat.append(np.nan)
        if len(x)>1:
            second_cat.append(x[1]['alias'])
        else:
            second_cat.append(np.nan)
        if len(x)>2:
            third_cat.append(x[2]['alias'])
        else:
            third_cat.append(np.nan)
    final["Restaurant Category 1"] = pd.Series(first_cat)
    final["Restaurant Category 2"] = pd.Series(second_cat)
    final["Restaurant Category 3"] = pd.Series(third_cat)
    pickup_list = []
    delivery_list = []
    reserve_list = []
    for entry in range(len(final['transactions'])):
        y = ast.literal_eval(final['transactions'][entry])
        if "pickup" in y:
            pickup_list.append(1)
        else:
            pickup_list.append(0)
        if "delivery" in y:
            delivery_list.append(1)
        else:
            delivery_list.append(0)
        if "restaurant_reservation" in y:
            reserve_list.append(1)
        else:
            reserve_list.append(0)
    final['Pickup Available'] = pd.Series(pickup_list)
    final['Delivery Available'] = pd.Series(delivery_list)
    final["Reservation Available"] = pd.Series(reserve_list)
    my_prices = []
    for entry in range(len(final['price'])):
        if final['price'][entry]=="$":
            my_prices.append("Low")
        elif final['price'][entry]=="$$":
            my_prices.append("Medium")
        elif final['price'][entry]=="$$$":
            my_prices.append("High")
        elif final['price'][entry]=="$$$$":
            my_prices.append("Very High")
        else:
            my_prices.append(np.nan)
    final['Price Level'] = pd.Series(my_prices)
    num_photos = []
    for entry in range(len(final['photos'])):
        x = ast.literal_eval(final['photos'][entry])
        x = len(x)
        num_photos.append(x)
    final['number of photos'] = pd.Series(num_photos)
    city = []
    state = []
    zip = []
    for entry in range(len(final['location'])):
        x= ast.literal_eval(final['location'][entry])
        y = x['city']
        z = x['state']
        a = x['zip_code']
        city.append(y)
        state.append(z)
        zip.append(a)
    final['City'] = pd.Series(city)
    final['State'] = pd.Series(state)
    final['Zip Code'] = pd.Series(zip)
    start_time = []
    end_time = []
    for entry in range(len(final['hours'])):
        if type(final['hours'][entry]) == float:
            start_time.append(np.nan)
            end_time.append(np.nan)
        else:
            x = ast.literal_eval(final['hours'][entry])
            start_time.append(x[0]['open'][0]['start'])
            end_time.append(x[0]['open'][0]['end'])
    final['Opening Time'] = pd.Series(start_time)
    final['Closing Time'] = pd.Series(end_time)
    
    messaging = []
    for entry in range(len(final['messaging'])):
        if type(final['messaging'][entry])!=float:
            messaging.append(1)
        else:
            messaging.append(0)
    final["Has Messaging"] = pd.Series(messaging)
    keep = ['id','is_closed',
    'is_claimed',
    'review_count',
    'rating',
    'Restaurant Category 1',
    'Restaurant Category 2',
    'Restaurant Category 3',
    'Pickup Available',
    'Delivery Available',
    "Reservation Available",
    'Price Level',
    'number of photos',
    'City',
    'State',
    'Zip Code',
    'Opening Time',
    'Closing Time',
    'Has Messaging']
    final = final[final.columns[final.columns.isin(keep)]]
    #final = final.dropna(how = 'any')
    #final['is_closed'] = np.where(final['is_closed']==False,0,1)

    return final

In [3]:
big_frisco = pd.read_csv("./RestruantData/All_Available_Frisco_Data_MoreINFO.csv")
big_enterprise = pd.read_csv("./RestruantData/All_Available_Enterprise_Data_MoreINFO.csv")
test = clean_yelp(big_frisco,big_enterprise)
test

,id,is_claimed,is_closed,review_count,rating,Restaurant Category 1,Restaurant Category 2,Restaurant Category 3,Pickup Available,Delivery Available,Reservation Available,Price Level,number of photos,City,State,Zip Code,Opening Time,Closing Time,Has Messaging
0,9VqZZiaiyGH_6smpvUKaQQ,True,False,127,4.5,bbq,breakfast_brunch,NaN,0,0,0,Medium,3,Aubrey,TX,76227,0700,2100,0
1,dPaJzxW0foPByEXbPFkh_Q,True,False,141,4.0,thai,NaN,NaN,1,1,0,Medium,3,Aubrey,TX,76227,1100,2130,0
2,RhL6hXqiz_nMz06Xx7tBcg,True,False,8,4.5,soulfood,foodtrucks,NaN,1,1,0,NaN,3,East Aubrey,TX,76227,1500,2100,0
3,yawhLqszfIp9q1aGNdLoig,True,False,23,4.5,foodtrucks,tacos,NaN,1,1,0,Medium,3,Aubrey,TX,76227,1100,1800,0
4,g_XBHNmwfPUOI4qE_bBgPQ,True,False,83,3.0,japanese,sushi,NaN,1,1,0,NaN,3,Aubrey,TX,76227,1100,1430,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565,VZAgmIhRGod34tfFkfww7g,True,False,247,3.5,tradamerican,sportsbars,NaN,0,0,0,Medium,1,Henderson,NV,89044,1130,0700,0
1566,XhUec_6-kgPy1_ZYUrzvPQ,True,False,433,4.0,steak,seafood,cocktailbars,0,1,0,High,3,Henderson,NV,89044,1700,2200,0
1567,0hFVE-lqnEytSsWZGevJDg,True,False,168,3.0,italian,desserts,NaN,0,1,0,Medium,3,Henderson,NV,89044,1700,2200,0
1568,x_eIYAOPCSM1BKvID2Vcbg,True,False,224,3.5,delis,sandwiches,tradamerican,0,1,0,Medium,2,Henderson,NV,89044,1100,2300,0


In [4]:
#take column and transform to column of list
#ast.literal_eval(frisco['categories'].iloc[0])

#big_df['categories'] = big_df['categories'].apply(lambda x :ast.literal_eval(x))
#create another "apply" with function that returns all values where "alias" is the key
#condense list into string and then split/expand the list
#allows us to be agnostic about the number of aliases

In [5]:
test.to_csv("Cleaned Frisco and Enterprise.csv")

In [6]:
test2 = test[test.columns[test.columns.isin(['Restaurant Category 1','Restaurant Category 2','Restaurant Category 3'])]]
test2 = test2.stack().reset_index(drop =True)

In [7]:
my_dict = {}
for entry in range(len(test2)):
    if test2[entry] not in my_dict.keys():
        my_dict[test2[entry]] = 1
    else:
         my_dict[test2[entry]] +=1

In [8]:
my_dict
len(test2.unique())

164